In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Imports and definitions

In [26]:
import numpy as np
import pandas as pd
from datetime import datetime
import xarray as xr
from pathlib import Path

In [27]:
from ecephys_analyses.data import channel_groups, paths, load
from ecephys.sglx_utils import load_timeseries
import ecephys.signal.timefrequency as tfr

In [28]:
xr.set_options(keep_attrs=True)

In [29]:
def parallel_spectrogram_welch(sig, **kwargs):
    freqs, spg_times, spg = tfr.parallel_spectrogram_welch(sig.values, sig.fs, **kwargs)
    spg_times = spg_times + sig.time.values.min()
    return xr.DataArray(
        spg,
        dims=("frequency", "time", "channel"),
        coords={"frequency": freqs, "time": spg_times, "channel": sig.channel.values},
        attrs={'units': f"{sig.units}^2/Hz", 'file_start': sig.fileCreateTime}
    )

In [30]:
def get_bandpower_from_spectrogram(spg, chans, f_range):
    bandpower = spg.sel(frequency=slice(*f_range)).sum(
        dim="frequency"
    )
    bandpower.attrs["freq_range"] = f_range
    bandpower.attrs['chans'] = chans

    return bandpower

In [31]:
def get_bandpower(sig, chans, f_range):
    # If no chans, return timeseries with negative power and approximately 1 sample per second. 
    if not chans:
        da = xr.DataArray(
            -1,
            dims=sig.time.dims,
            coords=sig.time.coords,
            attrs={"freq_range": f_range, "chans": chans},
        )
        return da[0 : -1 : int(sig.fs)]
    
    spg = parallel_spectrogram_welch(sig).median(dim="channel")
    return get_bandpower_from_spectrogram(spg, chans, f_range)

In [34]:
# This should probably not compute cx power when sr chans is absent. 

def get_condition_bandpower(subject, experiment, condition):
    cx_chans = channel_groups.superficial_ctx[subject]
    wm_chans = channel_groups.white_matter[subject]
    
    bin_paths = paths.get_sglx_style_datapaths(subject, experiment, condition, "lf.bin")
    sr_chans_paths = paths.get_sglx_style_datapaths(subject, experiment, condition, "sr_chans.csv")
    power_paths = paths.get_sglx_style_datapaths(subject, experiment, condition, "bandpower.nc")
    
    for bin_path, sr_chans_path, power_path in zip(bin_paths, sr_chans_paths, power_paths):
        sr_chans_df = load.load_sr_chans(sr_chans_path)
        power_by_epoch = list()
        for epoch in sr_chans_df.itertuples():
            cx = load_timeseries(bin_path, cx_chans, start_time=epoch.start_time, end_time=epoch.end_time)
            wm = load_timeseries(bin_path, wm_chans, start_time=epoch.start_time, end_time=epoch.end_time)
            sr = load_timeseries(bin_path, epoch.sr_chans, start_time=epoch.start_time, end_time=epoch.end_time)

            cx_wm_ref = cx - wm.values
            cx_wm_ref.attrs['reference'] = wm.channel.values

            sr_wm_ref = sr - wm.values
            sr_wm_ref.attrs['reference'] = wm.channel.values

            delta = (0.5, 4)
            theta = (5, 10)
            power = xr.Dataset({'mpta_delta_cbm_ref': get_bandpower(cx, cx_chans, delta), 
                                'mpta_theta_cbm_ref': get_bandpower(cx, cx_chans, theta),
                                'sr_delta_cbm_ref': get_bandpower(sr, epoch.sr_chans, delta), 
                                'sr_theta_cbm_ref': get_bandpower(sr, epoch.sr_chans, theta),
                                'mpta_delta_wm_ref': get_bandpower(cx_wm_ref, cx_chans, delta),
                                'mpta_theta_wm_ref': get_bandpower(cx_wm_ref, cx_chans, theta),
                                'sr_delta_wm_ref': get_bandpower(sr_wm_ref, epoch.sr_chans, delta),
                                'sr_theta_wm_ref': get_bandpower(sr_wm_ref, epoch.sr_chans, theta)})
            power_by_epoch.append(power)
            
        power = xr.concat(power_by_epoch, dim="time")
        power.attrs['file_start'] = cx.fileCreateTime
        Path(power_path).parent.mkdir(parents=True, exist_ok=True) # Create parent directories if they do not already exist.
        power.to_netcdf(power_path)
        power.close()
        
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"{current_time}: Finished {str(bin_path)}")